In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import regularizers
import yfinance as yf

# Solicita ao usuário inserir o símbolo do ativo
ticker = input("Insira o símbolo do ativo em caixa alta (ex: BOVA11.SA): ")

# Obtendo os dados do ativo
data = yf.download(ticker, start='1980-01-01', end='2023-07-30')

# Pré-processamento dos dados
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Close', 'Volume']].values)

# Dividindo os dados em treinamento e teste
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

# Obtenção do último preço manualmente
last_price = float(input("Insira o último preço do ativo: "))

# Adicionando o último preço manualmente aos dados de teste
test_data[-1, 0] = last_price

# Preparando os dados para a rede neural
def create_dataset(dataset, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_steps - 1):
        a = dataset[i:(i + time_steps), :]
        dataX.append(a)
        dataY.append(dataset[i + time_steps, 0])  # Usaremos apenas o preço para o Y
    return np.array(dataX), np.array(dataY)

time_steps = 50  # Número de períodos anteriores a serem usados como entrada (uma semana)
train_X, train_Y = create_dataset(train_data, time_steps)
test_X, test_Y = create_dataset(test_data, time_steps)

# Construindo a rede neural com mais neurônios e camadas de regularização
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(time_steps, 2)),  # Dois inputs: preço e volume
    tf.keras.layers.LSTM(150, return_sequences=True, kernel_regularizer=regularizers.l2(0.001)), # Regularização L2
    tf.keras.layers.Dropout(0.2),  # Dropout para evitar overfitting
    tf.keras.layers.LSTM(150, kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinando a rede neural com mais épocas
model.fit(train_X, train_Y, epochs=50, batch_size=64, verbose=1)

# Fazendo previsões
test_predictions = model.predict(test_X)

# Comparando a previsão com o último preço real
last_real_price = test_data[-1, 0]
predicted_price_next_week = test_predictions[-1, 0]

# Incluindo o preço projetado na mensagem de saída
prediction_text = f"Preço Projetado do Ativo na Próxima Semana: {predicted_price_next_week:.2f}\n"

# Determinando se o ativo irá subir, cair ou permanecer neutro com base no volume
if predicted_price_next_week > last_real_price and test_data[-1, 1] > np.mean(test_data[:, 1]):
    prediction_text += "O ATIVO IRÁ SUBIR! (BULL MARKET)"
elif predicted_price_next_week < last_real_price and test_data[-1, 1] < np.mean(test_data[:, 1]):
    prediction_text += "O ATIVO IRÁ CAIR! (BEAR MARKET)"
else:
    prediction_text += "O ATIVO IRÁ PERMANECER NEUTRO!"

print(prediction_text)


Insira o símbolo do ativo em caixa alta (ex: BOVA11.SA): btc-usd


[*********************100%%**********************]  1 of 1 completed


Insira o último preço do ativo: 667057
Epoch 1/50
40/40 [==============================] - 14s 230ms/step - loss: 0.2473
Epoch 2/50
40/40 [==============================] - 9s 222ms/step - loss: 0.0958
Epoch 3/50
40/40 [==============================] - 8s 205ms/step - loss: 0.0422
Epoch 4/50
40/40 [==============================] - 9s 223ms/step - loss: 0.0211
Epoch 5/50
40/40 [==============================] - 9s 232ms/step - loss: 0.0113
Epoch 6/50
40/40 [==============================] - 8s 202ms/step - loss: 0.0071
Epoch 7/50
40/40 [==============================] - 9s 238ms/step - loss: 0.0046
Epoch 8/50
40/40 [==============================] - 10s 242ms/step - loss: 0.0035
Epoch 9/50
40/40 [==============================] - 8s 211ms/step - loss: 0.0028
Epoch 10/50
40/40 [==============================] - 10s 241ms/step - loss: 0.0024
Epoch 11/50
40/40 [==============================] - 9s 226ms/step - loss: 0.0023
Epoch 12/50
40/40 [==============================] - 10s 242ms/st